In [2]:
# parsers/news_parser.py

import sqlite3
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
import time
import re
from urllib.parse import urlparse
from datetime import datetime

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class NewsParser:
    """
    Класс для парсинга новостей с championat.com
    """
    
    def __init__(self):
        self.driver = None
        self.wait = None
        self.parsed_news = []  # Храним спаршенные новости в памяти
        
    def setup_driver(self):
        """Инициализация драйвера Chrome"""
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)  # Ожидание до 10 секунд
        
    def get_category_from_url(self, url):
        """Извлекает категорию спорта из URL"""
        try:
            parsed_url = urlparse(url)
            path_parts = parsed_url.path.strip('/').split('/')
            if len(path_parts) > 0:
                return path_parts[0]
            return None
        except Exception as e:
            logger.error(f"Ошибка извлечения категории из {url}: {e}")
            return None

    def clean_text(self, text):
        """Очищает текст от лишних пробелов и символов"""
        if not text:
            return ""
        # Убираем лишние пробелы и переносы строк
        cleaned = re.sub(r'\s+', ' ', text.strip())
        return cleaned

    def is_duplicate_news(self, url):
        """Проверяет, есть ли уже такая новость в списке спаршенных"""
        return any(news['url'] == url for news in self.parsed_news)

    def save_news_data(self, news_data):
        """Сохраняет новость в список (пока без БД)"""
        try:
            # Добавляем timestamp парсинга
            news_data['parsed_at'] = datetime.now().isoformat()
            
            # Сохраняем в память
            self.parsed_news.append(news_data)
            
            logger.info(f"✅ Новость добавлена: {news_data['title'][:50]}...")
            logger.info(f"📊 Всего спаршено: {len(self.parsed_news)} новостей")
            
            return len(self.parsed_news) - 1  # Возвращаем индекс
            
        except Exception as e:
            logger.error(f"Ошибка сохранения данных: {e}")
            return None

    def get_parsed_news(self):
        """Возвращает все спаршенные новости"""
        return self.parsed_news

    def export_to_json(self, filename="parsed_news.json"):
        """Экспортирует спаршенные новости в JSON файл"""
        import json
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.parsed_news, f, ensure_ascii=False, indent=2)
            logger.info(f"📁 Данные экспортированы в {filename}")
        except Exception as e:
            logger.error(f"Ошибка экспорта: {e}")

    def print_news_summary(self):
        """Выводит краткую сводку спаршенных новостей"""
        print("\n" + "="*80)
        print(f"📊 СВОДКА ПАРСИНГА: {len(self.parsed_news)} новостей")
        print("="*80)
        
        for i, news in enumerate(self.parsed_news, 1):
            print(f"\n{i}. {news['title']}")
            print(f"   🔗 {news['url']}")
            print(f"   📂 Категория: {news.get('category', 'Не определена')}")
            print(f"   📝 Текст: {len(news.get('text', ''))} символов")
            print(f"   🏷️  Теги: {len(news.get('tags', []))}")
            print(f"   🖼️  Изображения: {len(news.get('images', []))}")
            print(f"   🎬 Видео: {len(news.get('videos', []))}")
            
            if news.get('tags'):
                print(f"   Теги: {', '.join([tag['name'] for tag in news['tags'][:3]])}{'...' if len(news['tags']) > 3 else ''}")
        
        print("\n" + "="*80)

    def parse_article_details(self, news_url):
        """Парсит детали статьи по ее URL"""
        data = {
            'text': None,
            'tags': [],
            'images': [],
            'videos': [],
            'published_at': None
        }
        
        try:
            self.driver.get(news_url)
            time.sleep(3)  # Даем время загрузиться
            
            # --- Парсинг текста статьи (пробуем разные селекторы) ---
            text_selectors = [
                "article p",
                ".article-content p",
                ".news-content p", 
                ".content p",
                "[class*='article'] p",
                ".text p"
            ]
            
            for selector in text_selectors:
                try:
                    paragraphs = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if paragraphs:
                        full_text = " ".join([p.text for p in paragraphs if p.text.strip()])
                        if len(full_text) > 50:  # Минимальная длина текста
                            data['text'] = self.clean_text(full_text)
                            logger.info(f"✅ Текст найден селектором: {selector}")
                            break
                except NoSuchElementException:
                    continue
            
            # Если основной текст не найден, пробуем альтернативный подход
            if not data['text']:
                try:
                    # Ищем любой длинный текст на странице
                    all_p = self.driver.find_elements(By.TAG_NAME, "p")
                    texts = [p.text for p in all_p if len(p.text.strip()) > 30]
                    if texts:
                        data['text'] = self.clean_text(" ".join(texts))
                        logger.info("✅ Текст найден альтернативным способом")
                except:
                    logger.warning(f"❌ Не удалось найти текст для {news_url}")
            
            # --- Парсинг даты публикации ---
            date_selectors = [
                "[datetime]",
                ".article__date",
                ".news-item__date",
                ".date",
                "[class*='date']"
            ]
            
            for selector in date_selectors:
                try:
                    date_element = self.driver.find_element(By.CSS_SELECTOR, selector)
                    date_text = date_element.get_attribute("datetime") or date_element.text
                    if date_text:
                        data['published_at'] = self.parse_date(date_text)
                        break
                except NoSuchElementException:
                    continue
            
            # --- Парсинг тегов ---
            tags_selectors = [
                ".article__tags a",
                ".tags__items a", 
                ".tags a",
                "[class*='tag'] a"
            ]
            
            for selector in tags_selectors:
                try:
                    tag_elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for tag_el in tag_elements:
                        tag_text = self.clean_text(tag_el.text)
                        tag_url = tag_el.get_attribute("href")
                        
                        if tag_text and tag_url and "теги" not in tag_text.lower():
                            data['tags'].append({
                                'name': tag_text,
                                'url': tag_url
                            })
                    
                    if data['tags']:
                        logger.info(f"✅ Найдено {len(data['tags'])} тегов")
                        break
                        
                except NoSuchElementException:
                    continue
            
            # --- Парсинг изображений ---
            try:
                img_elements = self.driver.find_elements(By.CSS_SELECTOR, "article img, .article-content img, .content img")
                
                for img in img_elements:
                    src = img.get_attribute("src")
                    alt = img.get_attribute("alt") or ""
                    
                    # Фильтруем служебные изображения
                    if src and not any(skip in src.lower() for skip in ['logo', 'icon', 'avatar', 'btn', 'sprite']):
                        data['images'].append({
                            'src': src,
                            'alt': self.clean_text(alt)
                        })
                        
                if data['images']:
                    logger.info(f"✅ Найдено {len(data['images'])} изображений")
                        
            except Exception as e:
                logger.warning(f"Ошибка парсинга изображений: {e}")
            
            # --- Парсинг видео ---
            try:
                iframe_elements = self.driver.find_elements(By.TAG_NAME, "iframe")
                for iframe in iframe_elements:
                    src = iframe.get_attribute("src")
                    if src and any(domain in src for domain in ["youtube.com", "rutube.ru", "vk.com/video"]):
                        data['videos'].append(src)
                        
                if data['videos']:
                    logger.info(f"✅ Найдено {len(data['videos'])} видео")
                        
            except Exception as e:
                logger.warning(f"Ошибка парсинга видео: {e}")
                
        except TimeoutException:
            logger.error(f"Таймаут при загрузке {news_url}")
        except WebDriverException as e:
            logger.error(f"Ошибка WebDriver при парсинге {news_url}: {e}")
        except Exception as e:
            logger.error(f"Неожиданная ошибка при парсинге {news_url}: {e}")
        
        return data

    def parse_date(self, date_string):
        """Парсит дату из различных форматов"""
        if not date_string:
            return None
            
        # Пытаемся распарсить разные форматы дат
        date_formats = [
            "%Y-%m-%dT%H:%M:%S",
            "%Y-%m-%d %H:%M",
            "%d.%m.%Y %H:%M",
            "%d.%m.%Y"
        ]
        
        for fmt in date_formats:
            try:
                return datetime.strptime(date_string.strip(), fmt).isoformat()
            except ValueError:
                continue
                
        logger.warning(f"Не удалось распарсить дату: {date_string}")
        return None

    def parse_news_page(self, page=1, limit=None):
        """Парсит страницу новостей"""
        url = f"https://www.championat.com/news/{page}.html"
        logger.info(f"Парсинг страницы: {url}")
        
        try:
            self.driver.get(url)
            time.sleep(3)  # Даем время загрузиться
            
            # Пытаемся найти контейнер с новостями разными способами
            posts = []
            selectors = [
                ".page-content .news-item",
                ".news-item",
                ".news-list .news-item",
                "[class*='news-item']"
            ]
            
            for selector in selectors:
                try:
                    posts = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if posts:
                        logger.info(f"✅ Найдены новости с селектором: {selector}")
                        break
                except:
                    continue
            
            if not posts:
                logger.error("❌ Не найдено новостей на странице")
                return
                
            if limit:
                posts = posts[:limit]
            
            logger.info(f"Найдено {len(posts)} новостей на странице")
            
            # Сначала собираем все ссылки и заголовки (избегаем stale elements)
            news_links = []
            for i, post in enumerate(posts, 1):
                try:
                    # Пробуем разные селекторы для ссылок
                    link_selectors = [
                        ".news-item__content a",
                        ".news-item__title a", 
                        "a[href*='/news/']",
                        "h3 a", "h2 a"
                    ]
                    
                    link_element = None
                    for sel in link_selectors:
                        try:
                            link_element = post.find_element(By.CSS_SELECTOR, sel)
                            break
                        except NoSuchElementException:
                            continue
                    
                    if link_element:
                        link = link_element.get_attribute("href")
                        title = self.clean_text(link_element.text)
                        
                        if link and title:
                            news_links.append({
                                'link': link,
                                'title': title,
                                'category': self.get_category_from_url(link)
                            })
                            logger.info(f"[{i}/{len(posts)}] Найдена: {title[:50]}...")
                    
                except Exception as e:
                    logger.warning(f"Не удалось извлечь данные из поста {i}: {e}")
                    continue
            
            logger.info(f"Собрано {len(news_links)} ссылок для парсинга")
            
            # Теперь парсим каждую новость отдельно
            parsed_count = 0
            skipped_count = 0
            
            for i, news_item in enumerate(news_links, 1):
                try:
                    # Проверяем дубликаты
                    if self.is_duplicate_news(news_item['link']):
                        logger.info(f"Пропускаем дубликат: {news_item['title'][:50]}...")
                        skipped_count += 1
                        continue
                    
                    logger.info(f"[{i}/{len(news_links)}] Парсинг детали: {news_item['title'][:50]}...")
                    
                    # Парсим детали статьи
                    article_data = self.parse_article_details(news_item['link'])
                    
                    # Добавляем основную информацию
                    article_data.update(news_item)
                    
                    # Сохраняем данные
                    if article_data['text']:
                        self.save_news_data(article_data)
                        parsed_count += 1
                    else:
                        logger.warning(f"Пустой текст для: {news_item['title']}")
                    
                    time.sleep(2)  # Задержка между запросами
                    
                except Exception as e:
                    logger.error(f"Ошибка при парсинге новости {i}: {e}")
                    continue
            
            logger.info(f"Завершено. Спаршено: {parsed_count}, Пропущено: {skipped_count}")
            
            # Выводим сводку
            if parsed_count > 0:
                self.print_news_summary()
            
        except TimeoutException:
            logger.error("Таймаут при загрузке страницы новостей")
        except Exception as e:
            logger.error(f"Общая ошибка парсинга: {e}")

    def run(self, pages=1, limit_per_page=None):
        """Основной метод запуска парсера"""
        try:
            self.setup_driver()
            logger.info("🚀 Запуск парсера новостей championat.com")
            
            for page in range(1, pages + 1):
                logger.info(f"📄 Парсинг страницы {page}")
                self.parse_news_page(page, limit_per_page)
                
                if page < pages:
                    time.sleep(3)  # Пауза между страницами
            
            # Итоговая статистика
            logger.info(f"🎉 Парсинг завершен! Всего новостей: {len(self.parsed_news)}")
            
            # Сохраняем в JSON для дальнейшего использования
            if self.parsed_news:
                self.export_to_json()
                    
        except KeyboardInterrupt:
            logger.info("Парсинг прерван пользователем")
        except Exception as e:
            logger.error(f"Критическая ошибка: {e}")
        finally:
            if self.driver:
                self.driver.quit()
                logger.info("🔚 Драйвер закрыт")

# Функция для запуска с разными настройками
def main():
    parser = NewsParser()
    
    # Можешь менять параметры для тестирования:
    # parser.run(pages=1, limit_per_page=3)  # 3 новости с 1 страницы
    # parser.run(pages=2, limit_per_page=5)  # 5 новостей с 2 страниц
    
    parser.run(pages=1, limit_per_page=5)  # Парсим 5 новостей с первой страницы

if __name__ == "__main__":
    main()

2025-09-01 20:15:14,563 - INFO - 🚀 Запуск парсера новостей championat.com
2025-09-01 20:15:14,564 - INFO - 📄 Парсинг страницы 1
2025-09-01 20:15:14,565 - INFO - Парсинг страницы: https://www.championat.com/news/1.html
2025-09-01 20:15:23,007 - INFO - ✅ Найдены новости с селектором: .page-content .news-item
2025-09-01 20:15:23,008 - INFO - Найдено 5 новостей на странице
2025-09-01 20:15:23,077 - INFO - [1/5] Найдена: Гаранин: «Балтика» выдала обалденный старт, они не...
2025-09-01 20:15:23,127 - INFO - [2/5] Найдена: Кристапс Порзингис, ЧЕ-2025: статистика в матче По...
2025-09-01 20:15:23,176 - INFO - [3/5] Найдена: Тренер «Лейкерс» рассказал, как именно Леброн Джей...
2025-09-01 20:15:23,232 - INFO - [4/5] Найдена: В Китае детей заставляют играть в игры по 15 часов...
2025-09-01 20:15:23,276 - INFO - [5/5] Найдена: 21 очко Порзингиса помогло сборной Латвии победить...
2025-09-01 20:15:23,279 - INFO - Собрано 5 ссылок для парсинга
2025-09-01 20:15:23,282 - INFO - [1/5] Парсинг детали: 


📊 СВОДКА ПАРСИНГА: 1 новостей

1. Гаранин: «Балтика» выдала обалденный старт, они не подстраиваются даже под грандов


2025-09-01 20:15:33,870 - INFO - 🔚 Драйвер закрыт
